In [1]:
print('hello world')

hello world


In [2]:
import os
from dotenv import load_dotenv
load_dotenv(override=True)
# LANGCHAIN_API_KEY = os.getenv('LANGCHAIN_API_KEY')

os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
 

print(os.environ['LANGCHAIN_API_KEY'])

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


# Indexing

In [3]:
from langchain.document_loaders import DirectoryLoader, TextLoader

folder_path = 'data/'

loader = DirectoryLoader(
    folder_path,
    glob="**/*.html",  # Modifier l'extension si vos fichiers ne sont pas en .xml
    loader_cls=TextLoader,
    recursive=True
)
documents = loader.load()
print(f"Nombre de documents chargés : {len(documents)}")

Nombre de documents chargés : 6154


In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def preprocess(doc):
    from bs4 import BeautifulSoup
    soup = BeautifulSoup(doc.page_content, 'html')
    text = soup.get_text()
    doc.page_content = text
    return doc

documents = [preprocess(doc) for doc in documents]

# Diviser les documents en morceaux plus petits si nécessaire
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2500,  # Taille des chunks en nombre de caractères
    chunk_overlap=200  # Chevauchement entre les chunks
)

splits = text_splitter.split_documents(documents)
print(f"Nombre de chunks après découpage : {len(splits)}")

Nombre de chunks après découpage : 38300


In [5]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# Spécifiez le répertoire où vous souhaitez stocker la base de données
persist_directory = 'chroma_db'

# Créez le vectorstore avec le paramètre persist_directory
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=OpenAIEmbeddings(),
                                    persist_directory=persist_directory)

# Sauvegardez le vectorstore sur le disque
vectorstore.persist()

retriever = vectorstore.as_retriever(search_kwargs={"k": 1})


/tmp/ipykernel_489675/2987431449.py:13: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


# Retrieval 

In [6]:
docs = retriever.get_relevant_documents("Si je roule 10000 kms en an combien je vais toucher d'ik avec un véhicule de 8 chevaux fiscaux")

print(docs)

/tmp/ipykernel_489675/3402203661.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents("Si je roule 10000 kms en an combien je vais toucher d'ik avec un véhicule de 8 chevaux fiscaux")


[Document(metadata={'source': 'data/BOFiP/documents/Contenu/Commentaire/AIS/13927-PGP/2024-07-10/data.html'}, page_content="voiture neuve comportant huit places assises. Ce véhicule est utilisé par M. W à la fois pour ses besoins professionnels et personnels. Il émet 200 grammes de CO2 par kilomètre.Le barème 2022 prévoit un tarif de 18\xa0188 €.Après application de l’abattement au bénéfice des familles nombreuses, pour un niveau d’émissions de CO2 égal à 140 grammes par kilomètre (200 CO2 g/km - 60 CO2 g/km), le montant de la taxe s’élèverait à 310 €.Après application de l’abattement pour les véhicules comportant au moins huit places, pour un niveau d’émissions de CO2 égal à 120 grammes par kilomètre (200 CO2 g/km - 80 CO2 g/km), le montant de la taxe serait nul.M. W pourra donc bénéficier d'un montant de taxe nul lors de l’immatriculation du véhicule.C. Véhicules recourant à des sources d'énergies alternatives1. Véhicules recourant exclusivement à l'électricité ou à l'hydrogène150Son

# Generation

In [7]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

# Prompt
template = """Réponds aux questions
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Answer the question based only on the following context:\n{context}\n\nQuestion: {question}\n'), additional_kwargs={})])

In [8]:
# LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [9]:
from langchain import hub
prompt_hub_rag = hub.pull("rlm/rag-prompt")

In [11]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("Si je roule 10000 kms en an combien je vais toucher d'indemnité kilométriques avec un véhicule de 8 chevaux fiscaux")

"Pour un véhicule de 8 chevaux fiscaux et un kilométrage de 10 000 km, vous toucherez une indemnité kilométrique en fonction du barème applicable pour l'imposition des revenus de l'année 2022. Vous devrez vous référer aux tranches relatives à des distances annuelles parcourues à titre professionnel entre 5 001 et 20 000 km pour connaître le coût kilométrique correspondant."